In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import metrics
import torch
import scipy.stats as stats
import researchpy as rp

Q1

In [3]:
pd.set_option('display.max_rows', 500)
df_raw = pd.read_excel('recruitment.xls')
df = df_raw.replace({"Gender": {1: "Male", 2: "Female"},
"BAMEyn": {1: "Black, Asian or Minority Ethnic", 2: "No"},
"FemaleONpanel": {1: "Male Only Panel", 2: "Female Member On Panel"}})
# 
# Number of Male Applicants and Number of Female Applicants 
df_raw.Gender.value_counts()
df_raw.BAMEyn.value_counts()
# UK discrimnatory Laws to determine gender and 
pd.DataFrame(df.groupby(["Gender"])["ShortlistedNY"].value_counts(normalize=True).sort_index())

ShortlistedNY
Gender ShortlistedNY               
Female 0                   0.752475
       1                   0.247525
Male   0                   0.512821
       1                   0.487179

Finding Percentages of Offer

In [4]:
pd.DataFrame(df.groupby(["Gender"])["OfferNY"].value_counts().sort_index())


OfferNY
Gender OfferNY         
Female 0.0           18
       1.0           10
Male   0.0            9
       1.0           18

In [5]:
df_shortlist = df[df.ShortlistedNY == 1]
pd.DataFrame(df_shortlist.groupby(["Gender"])["Interviewed"].value_counts(normalize=True).sort_index())

Interviewed
Gender Interviewed             
Female 0               0.440000
       1               0.560000
Male   0               0.289474
       1               0.710526

Considering Gender as a sensitive attribute

In [6]:
#A lot more females than men
print(df_raw.Gender.value_counts())

# Gender and Shortlisting 
print(pd.DataFrame(df_raw.groupby(["Gender"])["ShortlistedNY"].value_counts(normalize=True).sort_index()))

df_shortlist = df_raw[df_raw.ShortlistedNY == 1]

#Shortlisted women getting interviews
print(pd.DataFrame(df_shortlist.groupby(["Gender"])["Interviewed"].value_counts(normalize=True).sort_index()))

#Inteviewed Women getting offers 
df_interview = df_shortlist[df_shortlist.Interviewed == 1]
print(pd.DataFrame(df_interview.groupby(["Gender"])["OfferNY"].value_counts(normalize=True).sort_index()))

#Interviewed Women Offer ratio based on Female panel
df_offer = df_interview[df_interview.OfferNY == 1]
print(pd.DataFrame(df_offer.groupby(["Gender"])["FemaleONpanel"].value_counts(normalize=True).sort_index()))

2    202
1     78
Name: Gender, dtype: int64
                      ShortlistedNY
Gender ShortlistedNY               
1      0                   0.512821
       1                   0.487179
2      0                   0.752475
       1                   0.247525
                    Interviewed
Gender Interviewed             
1      0               0.289474
       1               0.710526
2      0               0.440000
       1               0.560000
                 OfferNY
Gender OfferNY          
1      0.0      0.333333
       1.0      0.666667
2      0.0      0.642857
       1.0      0.357143
                      FemaleONpanel
Gender FemaleONpanel               
1      1.0                 0.555556
       2.0                 0.444444
2      1.0                 0.200000
       2.0                 0.800000


In [7]:
df_accept = df_offer[df_offer.AcceptNY == 1]
df_accept

,ApplicantCode,Gender,BAMEyn,ShortlistedNY,Interviewed,FemaleONpanel,OfferNY,AcceptNY,JoinYN
0,1,1,2,1,1,1.0,1.0,1.0,1.0
1,2,1,2,1,1,1.0,1.0,1.0,1.0
2,3,1,2,1,1,1.0,1.0,1.0,1.0
3,4,1,2,1,1,2.0,1.0,1.0,1.0
19,20,2,2,1,1,2.0,1.0,1.0,1.0
24,25,2,1,1,1,2.0,1.0,1.0,1.0
25,26,2,2,1,1,2.0,1.0,1.0,1.0
140,141,1,2,1,1,1.0,1.0,1.0,1.0
141,142,1,2,1,1,1.0,1.0,1.0,1.0
142,143,1,1,1,1,1.0,1.0,1.0,1.0


In [8]:
#Lets see what gender accpets the offer if acceptance rate low then maybe not great benefits for women
print(pd.DataFrame(df_offer.groupby(["Gender"])["AcceptNY"].value_counts(normalize=True).sort_index()))


print(pd.DataFrame(df_accept.groupby(["Gender"])["JoinYN"].value_counts(normalize=True).sort_index()))

#Lets see out of the people who accpeted if there are any diff in joining 
df_join = df_accept[df_accept.JoinYN == 1]

print(pd.DataFrame(df_accept.groupby(["Gender"])["JoinYN"].value_counts(normalize=True).sort_index()))

                 AcceptNY
Gender AcceptNY          
1      0.0       0.333333
       1.0       0.666667
2      0.0       0.400000
       1.0       0.600000
               JoinYN
Gender JoinYN        
1      1.0        1.0
2      1.0        1.0
               JoinYN
Gender JoinYN        
1      1.0        1.0
2      1.0        1.0


In [9]:
# pd.crosstab(index = df_raw["Gender"], columns = df["OfferNYd"])
df_raw = df_raw.fillna(0)


Finding out indirect Bias

Task 1.5

In [10]:
from scipy.stats import chi2_contingency


In [11]:

croostab = pd.crosstab(df_raw['Gender'],df_raw['ShortlistedNY'])
statistic, p, dof, expected = stats.chi2_contingency(croostab)
alpha = 0.05 

print(p)
if p <= alpha:
    print('Dependent : Reject H0')
else:
    print('Independence : Accept H0')


0.00019227016669192619
Dependent : Reject H0


In [12]:
croostab = pd.crosstab(df_raw['BAMEyn'],df_raw['ShortlistedNY'])
statistic, p, dof, expected = stats.chi2_contingency(croostab)
alpha = 0.05 

print(p)
if p <= alpha:
    print('Dependent : Reject H0')
else:
    print('Independence : Accept H0')

1.4720402657609345e-06
Dependent : Reject H0


It shows that all columns barring the senstive attributes Race and Gender are dependent on sensitive attributes, hence there is indirect prejudice

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

x = df_raw.iloc[:,:4]
y = df_raw.OfferNY

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3, stratify = y)

classsfier = LogisticRegression()

classsfier.fit(x_train,y_train)


LogisticRegression()

In [14]:
prediction = classsfier.predict(x_test)
print(metrics.confusion_matrix(y_test,prediction))
print(metrics.classification_report(y_test,prediction,digits=3))

[[74  2]
 [ 6  2]]
              precision    recall  f1-score   support

         0.0      0.925     0.974     0.949        76
         1.0      0.500     0.250     0.333         8

    accuracy                          0.905        84
   macro avg      0.713     0.612     0.641        84
weighted avg      0.885     0.905     0.890        84



Implementing Regularization

In [15]:
import scipy.stats as stats
import researchpy as rp
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics 
df_raw = pd.read_excel('recruitment.xls')
df_raw = df_raw.fillna(0)
df_raw = df_raw.drop(columns = ['ApplicantCode'])


X = df_raw.iloc[:,:4]
y = df_raw.OfferNY
y = y.values

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)
X_train_gender = X_train.Gender
X_train_gender = X_train_gender.values


# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_test1 = X_test
y_test1 = y_test
X_train = torch.from_numpy(X_train.astype(np.float32))
X_trainGender = torch.from_numpy(X_train_gender.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)


class LogisticRegression(nn.Module):
    def __init__(self, number_of_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(number_of_features, 1)

    def forward(self, x): # Forward for predicting values
        Prediction = torch.sigmoid(self.linear(x))
        return Prediction

    def evaluate(self):
        with torch.no_grad():
            y_predicted = model(X_test)
            y_predicted_cls = y_predicted.round()
            acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
        return acc

    def train2(self,num_epochs,optimizer,criterion):
        for epoch in range(num_epochs):
            y_pred = self.forward(X_train) 
            loss = criterion(y_pred, y_train) # Loss of classifier
            loss.backward() 
            optimizer.step()
            optimizer.zero_grad()

            if (epoch+1) % 10 == 0:
                print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

model = LogisticRegression(n_features) 
num_epochs = 1000
learning_rate = 0.1
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3) Training loop
model.train2(num_epochs,optimizer,criterion) 

#4) Evaulation
model.evaluate()


epoch: 10, loss = 0.6365
epoch: 20, loss = 0.4860
epoch: 30, loss = 0.3969
epoch: 40, loss = 0.3395
epoch: 50, loss = 0.3000
epoch: 60, loss = 0.2714
epoch: 70, loss = 0.2498
epoch: 80, loss = 0.2330
epoch: 90, loss = 0.2196
epoch: 100, loss = 0.2087
epoch: 110, loss = 0.1997
epoch: 120, loss = 0.1921
epoch: 130, loss = 0.1856
epoch: 140, loss = 0.1800
epoch: 150, loss = 0.1752
epoch: 160, loss = 0.1709
epoch: 170, loss = 0.1671
epoch: 180, loss = 0.1638
epoch: 190, loss = 0.1608
epoch: 200, loss = 0.1580
epoch: 210, loss = 0.1556
epoch: 220, loss = 0.1533
epoch: 230, loss = 0.1513
epoch: 240, loss = 0.1494
epoch: 250, loss = 0.1477
epoch: 260, loss = 0.1461
epoch: 270, loss = 0.1446
epoch: 280, loss = 0.1433
epoch: 290, loss = 0.1420
epoch: 300, loss = 0.1408
epoch: 310, loss = 0.1397
epoch: 320, loss = 0.1386
epoch: 330, loss = 0.1376
epoch: 340, loss = 0.1367
epoch: 350, loss = 0.1358
epoch: 360, loss = 0.1350
epoch: 370, loss = 0.1342
epoch: 380, loss = 0.1335
epoch: 390, loss = 0.

tensor(0.9048)

Adversary Debaising Implementation

In [16]:

df_raw = pd.read_excel('recruitment.xls')
df_raw = df_raw.fillna(0)
df_raw = df_raw.drop(columns = ['ApplicantCode'])


# 0) Prepare data
# bc = datasets.load_breast_cancer()
X = df_raw.iloc[:,:4]
y = df_raw.OfferNY
y = y.values

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)
X_train_gender = X_train.Gender
X_train_gender = X_train_gender.values
X_train = X_train.values
X_test = X_test.values



# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train = torch.tensor(X_train.astype(np.float32),requires_grad=True)
X_trainGender = torch.tensor(X_train_gender.astype(np.float32),requires_grad=True)
X_test = torch.tensor(X_test.astype(np.float32),requires_grad=True)
y_train = torch.tensor(y_train.astype(np.float32),requires_grad=True)
y_test = torch.tensor(y_test.astype(np.float32),requires_grad=True)
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)
# Model Logistic Regression

X_trainGender = X_trainGender.view(X_train_gender.shape[0], 1)

class LogisticRegression(nn.Module):
    def __init__(self, number_of_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(number_of_features, 1)

    def forward(self, x): # Forward for predicting values
        Prediction = torch.sigmoid(self.linear(x))
        return Prediction

    def evaluate(self):
        with torch.no_grad():
            y_predicted = model(X_test)
            y_predicted_cls = y_predicted.round()
            acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
        return acc

    def train2(self,num_epochs,optimizer,criterion):
        for epoch in range(num_epochs):

            y_pred = self.forward(X_train) 
            loss = criterion(y_pred, y_train) # Loss of classifier
            loss.backward()

            optimizer.step()

            optimizer.zero_grad()

            if (epoch+1) % 10 == 0:
                print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

def trainWithAdversary(x, y, sensitive, num_epochs, learning_rate=0.01):

    for e in range(num_epochs):

        #### Compute outputs ####
        probas = torch.tensor(model.forward(x), requires_grad= True)
        #### Compute labels ####
        # labels = torch.where(probas >= .5, 1, 0) # threshold function
        # labelsMat = torch.unsqueeze(labels, 1)
        # labelsMat = labelsMat.float()
        
        #### Forward pass adversary ####
        adversary_probas = adversary.forward(probas)
        
        ### Compute loss of Adversary ###
        # print(size(sensitive))
        lossAdversary = criterion(adversary_probas,sensitive)

        ##Compute loss of Regular classifier##
        lossClassifier = criterion(probas,y)

        # gradientAdv = torch.tensor([1.])
        # gradientClass = torch.tensor([1.,1.,1.,1.])
        #### Compute gradients ####

        classGradient  = x.grad
        advGRadient = torch.tensor(1, dtype=torch.float)

        # advGRadient = None
        lossAdversary.backward(retain_graph=True,gradient = advGRadient) # Gives gradient from labels to sensitive attribute 
        lossClassifier.backward(retain_graph=True)
        sensitive.retain_grad()

        nice = []
        for (grad,var) in enumerate(classGradient):
            unitAdverse = torch.nn.functional.normalize(classGradient[var.long()])
            grad -= torch.sum(grad * unitAdverse) * unitAdverse
            grad -= lossAdversary * advGRadient.item()
            nice.append((grad,var))

        classGradient = nice

        optimizer.step()
        optimizer.zero_grad()
        # optimizer.step()

        # optimizer2.step()

        # optimizer.zero_grad()

        # optimizer2.zero_grad()

        if (e+1) % 10 == 0:
            print(f'epoch: {e+1}, loss = {lossClassifier.item():.4f}')
            print(f'epoch: {e+1}, loss = {lossAdversary.item():.4f}')
    



        #### Update classifier weights ####
        # print(adversary.weights)
        # unit_adversary_grad = [nn.functional.normalize(var[0]) for var in adversary.weights]
        # new_grad = []
        # model.weights
        
        # for grad,var in enumerate(classfierWeight):
            # A = del W which is classifier weight 
            # B =  del W which is adversary . weights
            # That leaves the projection 
        # A = classifier_grad_w * cost
        # B = classfierWeight_grad * 
    
        # unitAdvesaryVector = torch.norm(ad_grad[grad])
        # grad -= torch.sum(grad * unitAdvesaryVector) * unitAdvesaryVector
        # grad -= cost * adversary.weights[var.long()]
        # new_grad.append((grad,var))
        # #Update weights for the classifier
        # classifier_grad_w = new_grad
        


        # if e % 3 == 0:
        #     print(labels)
        

# 1st instance or Model
model = LogisticRegression(n_features) 
adversary = LogisticRegression(1)
# 2nd Indtance 
# mode2 = LogisticRegression(n_features) # Advisory 

# 2) Loss and optimizer
num_epochs = 1000
learning_rate = 0.1
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer2 = torch.optim.SGD(adversary.parameters(),lr=learning_rate)

# 3) Training loop
model.train2(num_epochs,optimizer,criterion)

#4) Evaulation
model.evaluate()


trainWithAdversary(X_train,y_train,X_trainGender,1000)

epoch: 10, loss = 0.5012
epoch: 20, loss = 0.3996
epoch: 30, loss = 0.3377
epoch: 40, loss = 0.2966
epoch: 50, loss = 0.2676
epoch: 60, loss = 0.2461
epoch: 70, loss = 0.2295
epoch: 80, loss = 0.2164
epoch: 90, loss = 0.2058
epoch: 100, loss = 0.1970
epoch: 110, loss = 0.1896
epoch: 120, loss = 0.1834
epoch: 130, loss = 0.1780
epoch: 140, loss = 0.1733
epoch: 150, loss = 0.1692
epoch: 160, loss = 0.1656
epoch: 170, loss = 0.1623
epoch: 180, loss = 0.1594
epoch: 190, loss = 0.1568
epoch: 200, loss = 0.1544
epoch: 210, loss = 0.1523
epoch: 220, loss = 0.1503
epoch: 230, loss = 0.1485
epoch: 240, loss = 0.1468
epoch: 250, loss = 0.1453
epoch: 260, loss = 0.1438
epoch: 270, loss = 0.1425
epoch: 280, loss = 0.1413
epoch: 290, loss = 0.1401
epoch: 300, loss = 0.1391
epoch: 310, loss = 0.1380
epoch: 320, loss = 0.1371
epoch: 330, loss = 0.1362
epoch: 340, loss = 0.1353
epoch: 350, loss = 0.1345
epoch: 360, loss = 0.1338
epoch: 370, loss = 0.1331
epoch: 380, loss = 0.1324
epoch: 390, loss = 0.

/var/folders/7p/66tsh0jd16n29z6nlxd264n80000gn/T/ipykernel_39773/3853799357.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probas = torch.tensor(model.forward(x), requires_grad= True)


epoch: 20, loss = 0.1165
epoch: 20, loss = 1.9193
epoch: 30, loss = 0.1165
epoch: 30, loss = 1.9193
epoch: 40, loss = 0.1165
epoch: 40, loss = 1.9193
epoch: 50, loss = 0.1165
epoch: 50, loss = 1.9193
epoch: 60, loss = 0.1165
epoch: 60, loss = 1.9193
epoch: 70, loss = 0.1165
epoch: 70, loss = 1.9193
epoch: 80, loss = 0.1165
epoch: 80, loss = 1.9193
epoch: 90, loss = 0.1165
epoch: 90, loss = 1.9193
epoch: 100, loss = 0.1165
epoch: 100, loss = 1.9193
epoch: 110, loss = 0.1165
epoch: 110, loss = 1.9193
epoch: 120, loss = 0.1165
epoch: 120, loss = 1.9193
epoch: 130, loss = 0.1165
epoch: 130, loss = 1.9193
epoch: 140, loss = 0.1165
epoch: 140, loss = 1.9193
epoch: 150, loss = 0.1165
epoch: 150, loss = 1.9193
epoch: 160, loss = 0.1165
epoch: 160, loss = 1.9193
epoch: 170, loss = 0.1165
epoch: 170, loss = 1.9193
epoch: 180, loss = 0.1165
epoch: 180, loss = 1.9193
epoch: 190, loss = 0.1165
epoch: 190, loss = 1.9193
epoch: 200, loss = 0.1165
epoch: 200, loss = 1.9193
epoch: 210, loss = 0.1165
ep